In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
import gc
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import PIL
from PIL import ImageDraw
from sklearn.model_selection import StratifiedKFold

from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator

from keras.applications import Xception
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import layers, models, optimizers

In [31]:
warnings.filterwarnings(action='ignore')

K.image_data_format()

'channels_last'

In [32]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [33]:
os.listdir('/gdrive/My Drive/Colab Notebooks/crop')

['cache',
 'models',
 'json',
 'img_out',
 'raw_img',
 'train_read',
 'class.csv',
 '.ipynb_checkpoints',
 'test_real',
 'train_real.csv',
 'test_real.csv',
 'train.csv',
 'train']

In [ ]:
DATA_PATH = '/gdrive/My Drive/Colab Notebooks/crop'

In [ ]:
IMAGE_SIZE = 299 #Image_size는 사용하는 pretained 모델 별 추천하는 이미지 사이즈를 주기 위해
BATCH_SIZE = 4
EPOCHS = 100

In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
# 이미지 폴더 경로
TRAIN_IMG_PATH = os.path.join(DATA_PATH, 'train')

# CSV 파일 경로 !!
df_train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
df_class = pd.read_csv(os.path.join(DATA_PATH, 'class.csv'))

In [ ]:
def del_index(a):  # window는 생략!
    if '.DS_Store' in a:
        del a[a.index('.DS_Store')]
        print('.DS_Store 삭제')
    if '.ipynb_checkpoints' in a:
        del a[a.index('.ipynb_checkpoints')]
        print('checkpoints 삭제')

In [ ]:
train_list = os.listdir(TRAIN_IMG_PATH)
#test_list = os.listdir(TEST_IMG_PATH)

In [ ]:
del_index(train_list)
#del_index(test_list)

In [41]:
# Data 누락 체크
if set(list(df_train.img)) == set(train_list) :
    print("Train file 누락 없음!")
else : 
    print("Train file 누락")

#if set(list(df_test.img)) == set(test_list) :
#    print("Test file 누락 없음!")
#else : 
#    print("Test file 누락")

Train file 누락 없음!


In [42]:
# Data 갯수
print("Number of Train Data : {}".format(df_train.shape[0]))
#print("Number of Test Data : {}".format(df_test.shape[0]))
print("타겟 클래스 총 갯수 : {}".format(df_class.shape[0]))
print("Train Data의 타겟 종류 갯수 : {}".format(df_train['class'].nunique()))

cntEachClass = df_train["class"].value_counts(ascending=False)
# Data 정보
print("Class with most count : {}".format(cntEachClass.index[0]))
print("Most Count : {}".format(cntEachClass.max()))
print("Class with fewest count : {}".format(cntEachClass.min()))
print("Mean : {}".format(cntEachClass.mean()))

Number of Train Data : 710
타겟 클래스 총 갯수 : 19185
Train Data의 타겟 종류 갯수 : 355
Class with most count : 642900990
Most Count : 2
Class with fewest count : 2
Mean : 2.0


In [43]:
from sklearn.model_selection import train_test_split      #split 생략!

df_train["class"] = df_train["class"].astype('str')

df_train = df_train[['img','class']]

X_train = df_train
X_val = df_train

print(X_train.shape)
print(X_val.shape)
#print(df_test.shape)

(710, 2)
(710, 2)


In [ ]:
model_path = '../model/'
if not os.path.exists(model_path):
    os.mkdir(model_path)

In [ ]:
def get_callback(model_name, patient):
    ES = EarlyStopping(monitor = 'val_loss',
                      patience = patient,
                      mode = 'min',
                      verbose = 1)
    RR = ReduceLROnPlateau(monitor = 'val_loss',
                          factor = 0.5,
                          patience = patient / 2,
                          min_lr = 0.000001,
                          verbose = 1,
                          mode = 'min')
    MC = ModelCheckpoint(filepath = filepath,
                        monitor = 'val_loss',
                        verbose = 1, 
                        save_best_only = True,
                        mode ='min')
    return [ES,RR,MC]

In [ ]:
def get_model(model_name, iamge_size):
    base_model = Xception(weights='imagenet', include_top=False)
    #base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dropout(0.25))
    model.add(layers.Dense(355, activation='softmax'))
    model.summary()

    optimizer = optimizers.RMSprop(lr=0.0001)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc'])

    return model

In [47]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.models import Sequential, Model
from keras.layers import Dense,Dropout,Flatten,Activation, Conv2D, GlobalAveragePooling2D
# Parameter
#img_size = (224, 224)
#nb_train_samples = len(X_train)
#nb_validation_samples = len(X_val)
#nb_test_samples = len(df_test)
#epochs = 20
#batch_size = 32

# Define Generator config
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    horizontal_flip = True, 
    vertical_flip = False,
    zoom_range=0.2,
    fill_mode = 'nearest'
    #preprocessing_function=preprocess_input)
)

val_datagen = ImageDataGenerator(rescale = 1./255
#preprocessing_function=preprocess_input)
                                )
test_datagen = ImageDataGenerator(rescale = 1./255
#preprocessing_function=preprocess_input
                                )
# Make Generator

train_generator = train_datagen.flow_from_dataframe(
    dataframe=X_train, 
    directory=TRAIN_IMG_PATH,
    x_col = 'img',
    y_col = 'class',
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    seed=42,
    shuffle = True)
    
valid_generator = val_datagen.flow_from_dataframe(
    dataframe=X_val, 
    directory=TRAIN_IMG_PATH,
    x_col = 'img',
    y_col = 'class',
    target_size = (IMAGE_SIZE,IMAGE_SIZE),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=32,
    shuffle = True)


Found 710 validated image filenames belonging to 355 classes.
Found 710 validated image filenames belonging to 355 classes.


In [48]:
model_xception = get_model(Xception, IMAGE_SIZE)

filepath = "my_Xception_model_{val_acc:.2f}_{val_loss:.4f}.h5"

history = model_xception.fit_generator(train_generator,
                                    steps_per_epoch = len(X_train.index) / BATCH_SIZE,
                                    epochs = EPOCHS,
                                    validation_data = valid_generator,
                                    validation_steps = len(X_val.index) / BATCH_SIZE,
                                    verbose = 1,
                                    shuffle = False,
                                    callbacks = get_callback(model_xception, 4))
gc.collect()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, None, None, 2048)  20861480  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 2048)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 1024)              2098176   
_________________________________________________________________
dropout_7 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 355)               363875    
Total para

311

In [ ]:
from sklearn.metrics import f1_score
def micro_f1(y_true,y_pred):
    return f1_score(y_true, y_pred, average='micro')
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
def get_steps(num_samples,batch_size):
    if(num_samples %batch_size) > 0:
        return (num_samples // batch_size) +1
    else:
        return num_samples // batch_size

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
model_list = sorted([i for i in os.listdir() if "my_" in i])

In [ ]:
model_list

In [ ]:
model_xception.load_weights(model_list[-1])

In [ ]:
%%time
test_generator.reset()
prediction = model_xception.predict_generator(
    generator = test_generator,
    steps = len(df_test)/BATCH_SIZE,
    verbose=1
)

In [ ]:
predicted_class_indices=np.argmax(prediction, axis=1)

# Generator class dictionary mapping
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

submission = pd.read_csv(os.path.join(DATA_PATH, 'sample_submission.csv'))
submission["class"] = predictions
submission.to_csv("submission.csv", index=False)
submission

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Download CSV file", filename = "submission.csv"):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(submission)